<a href="https://colab.research.google.com/github/VenkateshDas/legal_text_retrieval_coliee/blob/master/downsample_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [ ]:
import os
import re
import ast
import sys
from gensim import corpora
from nltk.tokenize import word_tokenize
import pprint
from gensim import models
from gensim import similarities
import pandas as pd
import pickle
import nltk 
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#Read files

In [ ]:
file_path ='/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/'

query_dataframe = pd.read_pickle(file_path + 'cleaned_ground_truth.pkl')
civil_code_dataframe = pd.read_pickle(file_path + 'cleaned_civil_code.pkl')

basic_classification_dataframe = pd.read_csv("/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/basic_classification_dataset.csv",index_col=0)
classification_dataframe = pd.read_csv("/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/classification_dataset.csv",index_col=0)


In [ ]:
classification_dataframe_cor = classification_dataframe.loc[classification_dataframe['Label']==1]
classification_dataframe_incor = classification_dataframe.loc[classification_dataframe['Label']==0]

#Preprocess text

In [ ]:
import spacy 
sp = spacy.load('en_core_web_sm')
# dirty_text = civil_code_df['Article_description']

def clean_data(dirty_text):


    #legal stop words -> Negation words (no, not) are not included, itemised tokens such as i,ii,iii etc are included and the normal stopwords in english from NLTK are included.
    legal_stop_words = ['her', 'then', 'this', 'isn', 'he', "couldn't", 'couldn', 'into', 'against', 'after', 'while', 'for', 'him', 'few', 'on', 'she', 'what', 'ain', 'is', 'mightn', "she's", 'was', 'more', 'me', 'them', 'should', 'are', 'as', 'than', 'so', 'i', "weren't", "aren't", 'wasn', 'an', 'herself', "mustn't", 'most', 'before', 'himself', 'my', 'yourselves', 'by', 'haven', 'any', 'or', 'needn', 'those', 'of', 'out', 'been', 'we', 'am', 'theirs', 'which', 'doesn', 'because', 'were', "didn't", 'above', 'over', 'y', 'when', 'had', 'with', 'during', 'how', 'too', 'through', "shan't", 'under', 'didn', 've', 'having', 'below', 'all', 'shouldn', 'ours', 'now', "hadn't", 'but', 'have', 'll', 'these', "wouldn't", 'if', 'doing', 'our', 'each', 'their', 'itself', "haven't", 'again', 'about', 'did', 'down', 'myself', 'hers', 'who', 'aren', 'do', 'hadn', "doesn't", 'whom', 'shan', "should've", 'from', 'own', "wasn't", "won't", 'at', 'both', 'yours', 'such', 'between', "you're", 'and', 'm', 'a', 'up', "mightn't", 'other', 're', 's', 'wouldn', 'd', 'in', 'there', 'off', 'you', 'until', 'don', 'will', 'where', 'themselves', "you'd", 'its', 'o', 'further', 'very', 'only', 'has', 'why', 'does', 'ourselves', 'just', 'ma', 'weren', 'mustn', 'can', "that'll", "you'll", "needn't", 'be', "shouldn't", 'yourself', 'nor', 'that', "isn't", 'the', 't', "hasn't", 'here', 'same', 'to', "you've", 'no', 'they', 'once', 'your', 'it', "don't", "it's", 'being', 'some', 'hasn', 'his','ii','iii','iv','v']
    punctuation_list = ['!','"','#','$','%','&',"'",'(',')','*','+',',','-','.','/',':',';','<','=','>','?','@','[','\',',']','^','_','`','{','|','}','~']
    # clean_token_list = []
    # clean_lemma_list = []
    # clean_pos_list = []

    # for i in range(dirty_text.shape[0]):
    text = sp(dirty_text.lower())
    tokens_op = []
    # tokens = []
    # lemma = []
    # pos = []
    
    for token in text:
        if str(token) not in legal_stop_words and str(token) not in punctuation_list and str(token) != " " and str(token) != '':
            tokens_op.append(str(token))
            # tokens.append(str(token))
    # for token in tokens_op:
    #     lemma.append(str(token.lemma_))
    #     pos.append(str(token.pos_))

    # clean_token_list.append(tokens_op)
    # clean_lemma_list.append(lemma)
    # clean_pos_list.append(pos)

    return tokens_op #,clean_lemma_list,clean_pos_list

#Retreive articles

## Retreive similar articles

In [ ]:
def retrieve_using_tfidf(index, query_list, top_n):
    """
    This function searches the most similar texts to the query.
        :param index: gensim.similarities object
        :param query: a string
        :param top_n: how many variants it returns
        :param prints: if True returns the results, otherwise prints the results
        :returns: a list of tuples (matched_document_index, similarity_value)
    """
    
    result = []
    for i in query_list:

      query_bow_vec = dictionary_articles.doc2bow(i)
      similarities = index[query_bow_vec]  # get similarities between the query and all index documents
      similarities = [(x, i) for i, x in enumerate(similarities)]
      similarities.sort(key=lambda elem: -elem[0])  # sorting by similarity_value in decreasing order
      for score,doc_number in similarities[:top_n]:
          result.append(incorrect_instances_full.iloc[doc_number])

        
    return  result 
    

In [ ]:
results = []
top_n = 10

for i in range(query_dataframe.shape[0]):
    print(i+1)
    correct_instances_full = classification_dataframe_cor.loc[classification_dataframe_cor['Serial_Num']==i+1]
    incorrect_instances_full = classification_dataframe_incor.loc[classification_dataframe_incor['Serial_Num']==i+1]
    correct_instances = correct_instances_full['Sequence']
    incorrect_instances = incorrect_instances_full['Sequence']

    clean_correct_instance = correct_instances.apply(clean_data)
    clean_incorrect_instances = incorrect_instances.apply(clean_data)

    #create dictionary
    dictionary_articles = corpora.Dictionary(clean_incorrect_instances)
    #create corpus
    corpus_articles = [dictionary_articles.doc2bow(line) for line in clean_incorrect_instances]
    #create model
    tfidf_model = models.TfidfModel(corpus_articles, smartirs='ntc')
    corpus_tfidf_model = tfidf_model[corpus_articles]
    #index
    index_tfidf_model = similarities.SparseMatrixSimilarity(corpus_tfidf_model, num_features=len(dictionary_articles))
    #retrieve similar instances 
    result = retrieve_using_tfidf(index_tfidf_model,clean_correct_instance,10)

    for i in result:
        results.append(i)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
len(results)

9000

In [ ]:
results_file_path  = "/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/top_10_similar_irrel_articles.pkl"

In [ ]:
with open(results_file_path,'wb') as f:
    pickle.dump(results,f)

In [ ]:
results = pickle.load(open(results_file_path,'rb'))

In [ ]:
len(results)

9000

## Create dataset

In [ ]:
downsampled_classification_dataframe = classification_dataframe_cor.copy()
downsampled_classification_dataframe

,Serial_Num,ID,Sequence,Article_number,Label
5,1,H21-1-1,Acceptance made by a minor that received an o...,5,1
782,2,H21-1-2,"If a minor, after getting permission for carr...",6,1
1561,3,H21-1-3,The purchase of daily household items may not...,9,1
2341,4,H21-1-4,"If a person under curatorship performs, witho...",13,1
2403,4,H21-1-4,"If a person under curatorship performs, witho...",120,1
...,...,...,...,...,...
536202,691,H24-29-1,A plaintiff shall prove that the opponent has...,712,1
536977,692,H24-29-3,Parents of a person who died by a tort may cl...,711,1
537436,693,H24-30-1,In cases where an obligee A demands performan...,452,1
538444,694,H24-30-2,In the cases where delivery of the subject ma...,633,1


In [ ]:
for i in range(len(results)):
    downsampled_classification_dataframe = downsampled_classification_dataframe.append(results[i])

downsampled_classification_dataframe.shape

(9900, 5)

In [ ]:
downsampled_classification_dataframe

,Serial_Num,ID,Sequence,Article_number,Label
5,1,H21-1-1,Acceptance made by a minor that received an o...,5,1
782,2,H21-1-2,"If a minor, after getting permission for carr...",6,1
1561,3,H21-1-3,The purchase of daily household items may not...,9,1
2341,4,H21-1-4,"If a person under curatorship performs, witho...",13,1
2403,4,H21-1-4,"If a person under curatorship performs, witho...",120,1
...,...,...,...,...,...
539283,695,H24-30-3,In cases where A claims purchase money of mov...,689,0
539219,695,H24-30-3,In cases where A claims purchase money of mov...,632,0
539111,695,H24-30-3,In cases where A claims purchase money of mov...,537,0
539112,695,H24-30-3,In cases where A claims purchase money of mov...,538,0


In [ ]:
downsampled_classification_dataframe = downsampled_classification_dataframe.sort_values(by='Serial_Num', ascending=True)

In [ ]:
downsampled_classification_dataframe

,Serial_Num,ID,Sequence,Article_number,Label
5,1,H21-1-1,Acceptance made by a minor that received an o...,5,1
91,1,H21-1-1,Acceptance made by a minor that received an o...,135,0
77,1,H21-1-1,Acceptance made by a minor that received an o...,121_2,0
20,1,H21-1-1,Acceptance made by a minor that received an o...,20,0
87,1,H21-1-1,Acceptance made by a minor that received an o...,131,0
...,...,...,...,...,...
539134,695,H24-30-3,In cases where A claims purchase money of mov...,556,0
539283,695,H24-30-3,In cases where A claims purchase money of mov...,689,0
539219,695,H24-30-3,In cases where A claims purchase money of mov...,632,0
539111,695,H24-30-3,In cases where A claims purchase money of mov...,537,0


In [ ]:
downsampled_classification_dataframe.to_csv('/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/only_top10_similar_downsampled_classification_dataset.csv')

In [ ]:
up_and_downsampled_classification_dataframe = classification_dataframe_cor.copy() 

In [ ]:
''' Oversampling the Relevant class in the dataframe '''
N_times = 10
up_and_downsampled_classification_dataframe = pd.concat([up_and_downsampled_classification_dataframe]*N_times, ignore_index=True)

In [ ]:
up_and_downsampled_classification_dataframe

,Serial_Num,ID,Sequence,Article_number,Label
0,1,H21-1-1,Acceptance made by a minor that received an o...,5,1
1,2,H21-1-2,"If a minor, after getting permission for carr...",6,1
2,3,H21-1-3,The purchase of daily household items may not...,9,1
3,4,H21-1-4,"If a person under curatorship performs, witho...",13,1
4,4,H21-1-4,"If a person under curatorship performs, witho...",120,1
...,...,...,...,...,...
8995,691,H24-29-1,A plaintiff shall prove that the opponent has...,712,1
8996,692,H24-29-3,Parents of a person who died by a tort may cl...,711,1
8997,693,H24-30-1,In cases where an obligee A demands performan...,452,1
8998,694,H24-30-2,In the cases where delivery of the subject ma...,633,1


In [ ]:
for i in range(len(results)):
    up_and_downsampled_classification_dataframe = up_and_downsampled_classification_dataframe.append(results[i])

up_and_downsampled_classification_dataframe.shape

(18000, 5)

In [ ]:
up_and_downsampled_classification_dataframe.to_csv('/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/top10_similar_up_and_downsampled_classification_dataset.csv')

## Retreive Random articles

In [ ]:
N = 10 # articles per row
random_downsample_dataframe = pd.DataFrame()

for i in range(query_dataframe.shape[0]):
    print(i+1)
    correct_instances_full = classification_dataframe_cor.loc[classification_dataframe_cor['Serial_Num']==i+1]
    # print(i+1,correct_instances_full.shape[0])
    incorrect_instances_full = classification_dataframe_incor.loc[classification_dataframe_incor['Serial_Num']==i+1]
    n = N * correct_instances_full.shape[0]
    random_downsample = incorrect_instances_full.sample(n=n,replace=False)
    random_downsample_dataframe = random_downsample_dataframe.append(random_downsample)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
random_downsample_dataframe

,Serial_Num,ID,Sequence,Article_number,Label
332,1,H21-1-1,Acceptance made by a minor that received an o...,383,0
2,1,H21-1-1,Acceptance made by a minor that received an o...,3,0
396,1,H21-1-1,Acceptance made by a minor that received an o...,422_2,0
552,1,H21-1-1,Acceptance made by a minor that received an o...,522,0
321,1,H21-1-1,Acceptance made by a minor that received an o...,372,0
...,...,...,...,...,...
538544,695,H24-30-3,In cases where A claims purchase money of mov...,1,0
538721,695,H24-30-3,In cases where A claims purchase money of mov...,229,0
538860,695,H24-30-3,In cases where A claims purchase money of mov...,367,0
539040,695,H24-30-3,In cases where A claims purchase money of mov...,484,0


In [ ]:
''' Concatenate the Downsampled Relevant class and the Irrelevant class '''
random_downsample_classification_dataframe = pd.concat([classification_dataframe_cor,random_downsample_dataframe], ignore_index=True)

In [ ]:
random_downsample_classification_dataframe = random_downsample_classification_dataframe.sort_values(by='Serial_Num', ascending=True)

In [ ]:
random_downsample_classification_dataframe

,Serial_Num,ID,Sequence,Article_number,Label
0,1,H21-1-1,Acceptance made by a minor that received an o...,5,1
909,1,H21-1-1,Acceptance made by a minor that received an o...,466_5,0
908,1,H21-1-1,Acceptance made by a minor that received an o...,295,0
907,1,H21-1-1,Acceptance made by a minor that received an o...,680_2,0
906,1,H21-1-1,Acceptance made by a minor that received an o...,702,0
...,...,...,...,...,...
9894,695,H24-30-3,In cases where A claims purchase money of mov...,390,0
9895,695,H24-30-3,In cases where A claims purchase money of mov...,1,0
9896,695,H24-30-3,In cases where A claims purchase money of mov...,229,0
9897,695,H24-30-3,In cases where A claims purchase money of mov...,367,0


In [ ]:
random_downsample_classification_dataframe.to_csv('/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/top10_random_downsampled_classification_dataset.csv')

In [ ]:
''' Oversampling the Relevant class in the dataframe '''
N_times = 10
up_and_random_downsampled_classification_dataframe = pd.concat([classification_dataframe_cor]*N_times, ignore_index=True)

In [ ]:
up_and_random_downsampled_classification_dataframe =  pd.concat([up_and_random_downsampled_classification_dataframe,random_downsample_dataframe], ignore_index=True)

In [ ]:
up_and_random_downsampled_classification_dataframe = up_and_random_downsampled_classification_dataframe.sort_values(by='Serial_Num', ascending=True)

In [ ]:
up_and_random_downsampled_classification_dataframe.loc[up_and_random_downsampled_classification_dataframe['Serial_Num']==1]

,Serial_Num,ID,Sequence,Article_number,Label
0,1,H21-1-1,Acceptance made by a minor that received an o...,5,1
2700,1,H21-1-1,Acceptance made by a minor that received an o...,5,1
9009,1,H21-1-1,Acceptance made by a minor that received an o...,466_5,0
9008,1,H21-1-1,Acceptance made by a minor that received an o...,295,0
9007,1,H21-1-1,Acceptance made by a minor that received an o...,680_2,0
9006,1,H21-1-1,Acceptance made by a minor that received an o...,702,0
9005,1,H21-1-1,Acceptance made by a minor that received an o...,167,0
9004,1,H21-1-1,Acceptance made by a minor that received an o...,372,0
1800,1,H21-1-1,Acceptance made by a minor that received an o...,5,1
9003,1,H21-1-1,Acceptance made by a minor that received an o...,522,0


In [ ]:
up_and_random_downsampled_classification_dataframe

,Serial_Num,ID,Sequence,Article_number,Label
0,1,H21-1-1,Acceptance made by a minor that received an o...,5,1
2700,1,H21-1-1,Acceptance made by a minor that received an o...,5,1
9009,1,H21-1-1,Acceptance made by a minor that received an o...,466_5,0
9008,1,H21-1-1,Acceptance made by a minor that received an o...,295,0
9007,1,H21-1-1,Acceptance made by a minor that received an o...,680_2,0
...,...,...,...,...,...
2699,695,H24-30-3,In cases where A claims purchase money of mov...,555,1
1799,695,H24-30-3,In cases where A claims purchase money of mov...,555,1
899,695,H24-30-3,In cases where A claims purchase money of mov...,555,1
17998,695,H24-30-3,In cases where A claims purchase money of mov...,484,0


In [ ]:
up_and_random_downsampled_classification_dataframe.to_csv('/content/drive/My Drive/COLIEE Retrieval /COLIEE Dry Run Data /preprocessed_data/bert_data/top10_up_and_random_downsampled_classification_dataset.csv')